In [1]:
import multiprocessing as mp 
import time

In [4]:
def calc_cuad(numeros, result):
    for idx, n in enumerate(numeros):
        result[idx]=n*n
    print("Resultado del proceso:", result[:])

nums=range(10)
t=time.time()

result=mp.Array('i',10)

# Pipes

Los pipes o tuberias representan una forma de pasar informacion. Los tipos de pipes pueden ser anonimos o con nombre. Los pipes anonimos son metodos de comunicacion simple del tipo FIFO, son ultilizados para la comunicacion interproceso y solo transmiten en una sola direccion a la vez; para una comunicacion bidireccional (duplex) se requierem dos pipes anonimos. Los pipes con nombre son identicos a los anonimos con la diferencia que duraran siempre y cuando el sistema operativo tenga activos pipes anonimos.

In [1]:
from multiprocessing import Process, Pipe

def f(conn):
    conn.send(['hello world'])
    conn.close

if __name__ =='__main__':
    parent_conn, child_conn =Pipe()
    p=Process(target=f,args=(child_conn,))
    p.start()
    print(parent_conn.recv())
    p.join()

['hello world']


## Ejercicio 1

In [7]:
from multiprocessing import Process, Pipe

def funcion1(conn):
    conn.send(['hello'])
    conn.close

def funcion2(conn):
    conn.send(['world'])
    conn.close

if __name__ =='__main__':
    parent_conn, child_conn =Pipe()
    p1=Process(target=funcion1,args=(child_conn,))
    p2=Process(target=funcion2,args=(child_conn,))
    p1.start()
    p2.start()
    print(parent_conn.recv())
    print(parent_conn.recv())
    p1.join()
    p2.join()
    

['hello']
['world']


In [5]:
from multiprocessing import Process, Pipe

def worker(conn):
    print(conn.recv())
    conn.send("sent from child process")
    conn.close()

conn1, conn2 = Pipe()
process=Process(target=worker,args=(conn2,))
process.start()

conn1.send("sent from main process")
print(conn1.recv())
process.join()

sent from main process
sent from child process


## Ejercicio 2

In [6]:
from multiprocessing import Process, Pipe

def funcion1(conn):
    print(conn.recv())
    conn.send(['hello'])
    conn.close()

def funcion2(conn):
    print(conn.recv())
    conn.send(['world'])
    conn.close()

if __name__ =='__main__':
    parent_conn, child_conn =Pipe()
    p1=Process(target=funcion1,args=(child_conn,))
    p2=Process(target=funcion2,args=(child_conn,))
    p1.start()
    p2.start()
    parent_conn.send("sent from main process")
    parent_conn.send("sent from main process")
    print(parent_conn.recv())
    print(parent_conn.recv())    
    parent_conn.close()
    p1.join()
    p2.join()

sent from main processsent from main process

['hello']
['world']


In [7]:
nombres = ["Carlos","Renata", "Rebeca","Sandra","END"]

def send_msgs(conn,msgs):
    for msg in msgs:
        conn.send(msg)
    conn.close()
def recv_msgs(conn):
    while 1:
        msg=conn.recv()
        if msg == "END":
            break 
        print(msg)

conn1, conn2 = Pipe()
p1=Process(target=send_msgs,args=(conn1,nombres))
p2=Process(target=recv_msgs,args=(conn2,))

p1.start()
p2.start()

p1.join()
p2.join()

Carlos
Renata
Rebeca
Sandra


# Pipe os

In [8]:
import os, sys
import multiprocessing

class ChildProcess(multiprocessing.Process):

    def __init__(self,pipein):
        super(ChildProcess,self).__init__()
        self.pipein=pipein

    def run(self)

SyntaxError: invalid syntax (<ipython-input-8-2096b63928e3>, line 10)

# Colas 
La clase queue del modulo multiprocesing facilita la implementacion de una cola FIFO para procesos como para subprocesos. Cualquier objeto de Python se puede paar a traves de un objeto queue. 

In [9]:
from multiprocessing import Queue

def f(q):
    q.put([42,None,'hello'])

if __name__ == '__main__':
    q=Queue()
    p=Process(target=f, args=(q,))
    p.start()
    
    p.join()
    print(q.get())

[42, None, 'hello']


In [10]:
def calc_cuad(numeros, q):
    for n in numeros:
        q.put(n*n)
if __name__ == '__main__':
    nums=range(10)
    q=Queue()
    p=Process(target=calc_cuad, args=(nums,q))
    p.start()
    p.join()
while not q.empty():
    print(q.get())


0
1
4
9
16
25
36
49
64
81


In [11]:
import multiprocessing

class MyWorker():
    def __init__(self,x):
        self.x=x
    def process(self):
        pname=multiprocessing.current_process().name
        print('Starting process %s for number %i...'%(pname,self.x))
def work(q):
    worker = q.get()
    worker.process()

if __name__ == '__main__':
    my_queue=multiprocessing.Queue()
    p=multiprocessing.Process(target=work,args=(my_queue,))
    p.start()

    my_queue.put(MyWorker(1))

    my_queue.close()
    my_queue.join_thread()
    p.join()

    print('Done.')


Starting process Process-8 for number 1...
Done.


# Bloqueo

Existen recursos los cuales no se deben acceder al mismo tiempo por dos procesos o mas; por lo que es necesario proteger o bloquear el acceso a estos recursos compartidos: memoria, archivos, bases de datos.

In [42]:
import time
import multiprocessing

def deposit(balance):
    for i in range(100):
        time.sleep(0.01)
        balance.value=balance.value + 1
def withdraw(balance):
    for i in range(100):
        time.sleep(0.01)
        balance.value=balance.value - 1

if __name__ == '__main__':
    balance = multiprocessing.Value('i',200)
    d=multiprocessing.Process(target=deposit, args=(balance,))
    w=multiprocessing.Process(target=withdraw, args=(balance,))
    d.start()
    w.start()
    d.join()
    w.join()
    print(balance.value)

200


In [44]:
import time
import multiprocessing

def deposit(balance,lock):
    for i in range(100):
        time.sleep(0.01)
        lock.acquire()
        balance.value=balance.value + 1
        lock.release()
def withdraw(balance,lock):
    for i in range(100):
        time.sleep(0.01)
        lock.acquire()
        balance.value=balance.value - 1
        lock.release()
if __name__ == '__main__':
    balance = multiprocessing.Value('i',200)
    lock = multiprocessing.Lock()
    d = multiprocessing.Process(target=deposit, args=(balance,lock))
    w = multiprocessing.Process(target=withdraw, args=(balance,lock))
    d.start()
    w.start()
    d.join()
    w.join()
    print(balance.value)

200
